In [1]:
!nvidia-smi

Sat Feb 25 06:21:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:0B:00.0 Off |                    0 |
| N/A   30C    P0    43W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup environment

In [12]:
workspace_dir = '/nfs/Workspace/TeethSeg'
%cd {workspace_dir}
%load_ext autoreload
%autoreload 2

/nfs/Workspace/TeethSeg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess

In [14]:
!PYTHONPATH={workspace_dir} /opt/conda/envs/TeethSeg/bin/python {workspace_dir}/expers/preprocess.py

fold: 2
train files (4): ['1001162439_20150708', '1001275319_20180114', '1001382496_20180423', '1001382496_20201206']
val files (2): ['1001470164_20180114', '1001487462_20180109']
test files (2): ['1001487462_20180527', '1001487462_20190427']
save json to /nfs/Workspace/dataset/teeth/data_29/data.json


## Setup config

In [50]:
import os

model_name = 'swinunetr'
data_name = 'teeth'
exp_name = 'exp_2'

root_exp_dir = os.path.join(
    workspace_dir, 
    'exps',
    model_name,
    data_name,
    exp_name
)


data_dir = '/nfs/Workspace/dataset/teeth/data_29'
data_json = os.path.join(data_dir, 'data.json')
model_dir = os.path.join(root_exp_dir, 'models')
log_dir = os.path.join(root_exp_dir, 'logs')
eval_dir = os.path.join(root_exp_dir, 'evals')

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')


os.makedirs(f'{workspace_dir}/exps/{model_name}/{data_name}', exist_ok=True)

%cd {workspace_dir}/exps/{model_name}/{data_name}

/nfs/Workspace/TeethSeg/exps/swinunetr/teeth


## Train/Test

In [52]:
# training
!PYTHONPATH={workspace_dir} /opt/conda/envs/TeethSeg/bin/python {workspace_dir}/expers/train.py \
--exp_name={exp_name} \
--data_name={data_name} \
--root_exp_dir={root_exp_dir} \
--model_name={model_name}\
--data_dir={data_dir} \
--model_dir={model_dir} \
--log_dir={log_dir} \
--eval_dir={eval_dir} \
--data_dicts_json={data_json} \
--split_train_ratio=0.6 \
--num_fold=3 \
--fold=2 \
--val_every=20 \
--max_early_stop_count=20 \
--max_epoch=2000  \
--pin_memory \
--out_channels=29 \
--scale_intensity_type='range' \
--a_min=218 \
--a_max=3187 \
--roi_x=96 \
--roi_y=96 \
--roi_z=96 \
--space_x=1.0 \
--space_y=1.0 \
--space_z=1.0 \
--optim='AdamW' \
--lr=1e-4 \
--weight_decay=5e-4 \
--checkpoint={final_checkpoint} \
# --test_mode

cuda is available
<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
use scale intensity by range
use scale intensity by range
load json from /nfs/Workspace/dataset/teeth/data_29/data.json
train files (4): ['1001162439_20150708', '1001275319_20180114', '1001382496_20180423', '1001382496_20201206']
val files (2): ['1001470164_20180114', '1001487462_20180109']
test files (2): ['1001487462_20180527', '1001487462_20190427']

load test dataset ...
Loading dataset: 100%|████████████████████████████| 2/2 [00:04<00:00,  2.25s/it]
model: swinunetr
optimzer: AdamW
{'lr': 0.0001, 'weight_decay': 0.0005}
=> loaded checkpoint '/nfs/Workspace/TeethSeg/exps/swinunetr/teeth/exp_2/models/final_model.pth' (epoch 821) (bestacc 0.37300512194633484) (early stop count 20)
eval (0 / 2 Steps):  50%|████████████▌            | 1/2 [00:04<00:04,  4.14s/it]the prediction of 